In [ ]:
import pandas as pd
import sentencepiece as spm
import torch
from torch.nn.utils.rnn import pad_sequence
from torch import nn, optim 
from torch.utils.data import DataLoader, TensorDataset

PATH = 'C:\Emotion Classification\data\\full_dataset\goemotions_1.csv'
TOKENIZER = 'C:\Emotion Classification\\traning\\vocab\\tokenizer.model'

sp = spm.SentencePieceProcessor()
sp.load(TOKENIZER)

batchsize = 100

<>:8: SyntaxWarning: invalid escape sequence '\E'
<>:9: SyntaxWarning: invalid escape sequence '\E'
<>:8: SyntaxWarning: invalid escape sequence '\E'
<>:9: SyntaxWarning: invalid escape sequence '\E'
C:\Users\illya\AppData\Local\Temp\ipykernel_25204\35637546.py:8: SyntaxWarning: invalid escape sequence '\E'
  PATH = 'C:\Emotion Classification\data\\full_dataset\goemotions_1.csv'
C:\Users\illya\AppData\Local\Temp\ipykernel_25204\35637546.py:9: SyntaxWarning: invalid escape sequence '\E'
  TOKENIZER = 'C:\Emotion Classification\\traning\\vocab\\tokenizer.model'


In [3]:
df = pd.read_csv(PATH, sep=',')

In [4]:
df.columns

Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

In [5]:
df = df.drop(columns=['id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id'])
df['example_very_unclear'] = df['example_very_unclear'].astype(int)

In [6]:
split_index = int(len(df) * 0.85)
split_index

59500

In [7]:
def pipeline_text(data):
    data = data.copy()
    
    data["text"] = data["text"].apply(lambda x: sp.encode(x, out_type=int))
    y = data.drop(columns=['text'])
    
    list_of_lists = data['text'].tolist()
    tensor_list = [torch.tensor(seq, dtype=torch.long) for seq in list_of_lists]
    X = pad_sequence(tensor_list, batch_first=True, padding_value=0)
    y = torch.tensor(y.values, dtype=torch.int64)
    
    X_train = X[:split_index]
    X_test = X[split_index:]
    
    y_train = y[:split_index]
    y_test = y[split_index:]
    
    
    return X_train, y_train, X_test, y_test
    
X_train, y_train, X_test, y_test = pipeline_text(df)

In [8]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, type(X_train), type(y_train), type(X_test), type(y_test)


(torch.Size([59500, 142]),
 torch.Size([59500, 29]),
 torch.Size([10500, 142]),
 torch.Size([10500, 29]),
 torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 torch.Tensor)

In [25]:
sequence_len = X_train.shape[1]
input_len = X_train.shape[0]
hidden_size = 128
num_layers = 2
num_classes = y_train.shape[1]
num_epochs = 5
learning_rate = 0.01

In [26]:
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_classes) 
    
    def forward(self, X):
        hidden_size = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        cell_state = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        out, _ = self.lstm(X, (hidden_size, cell_state))
        out = self.output_layer(out[:, -1, :])
        return out
    

In [27]:
model = LSTM(input_len, hidden_size, num_layers, num_classes)

In [28]:
model

LSTM(
  (lstm): LSTM(59500, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=29, bias=True)
)

In [31]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train(num_epochs, model, train_dataloader):